Thanks to [Jirka Borovec](https://www.kaggle.com/jirkaborovec) majority of the code is copied from his [notebook](https://www.kaggle.com/code/jirkaborovec/tract-segm-eda-baseline-flash-deeplab-v3). please let me know in the comments in case of any bugs.

In [ ]:
!pip install -q https://github.com/Borda/kaggle_image-segm/archive/refs/heads/main.zip

In [ ]:
import os, glob
import pandas as pd
import matplotlib.pyplot as plt

DATASET_FOLDER = "/kaggle/input/uw-madison-gi-tract-image-segmentation"
path_csv = os.path.join(DATASET_FOLDER, "train.csv")
df_train = pd.read_csv(path_csv)

In [ ]:
all_imgs = glob.glob(os.path.join(DATASET_FOLDER, "train", "case*", "case*_day*", "scans", "*.png"))
all_imgs = [p.replace(DATASET_FOLDER, "") for p in all_imgs]

print(f"images: {len(all_imgs)}")
print(f"annotated: {len(df_train['id'].unique())}")

In [ ]:
from kaggle_imsegm.data import extract_tract_details

df_train[['Case','Day','Slice', 'image', 'image_path','height','width']] = df_train['id'].apply(
    lambda x: pd.Series(extract_tract_details(x, DATASET_FOLDER))
)

df_train = df_train[~df_train.segmentation.isna()]

display(df_train.head())

In [ ]:
from tqdm.auto import tqdm

train_ = []
for id_, dfg in tqdm(df_train.groupby("id")):
    row = dict(dfg.iloc[0])
    for _, (cls, segm) in dfg[["class", "segmentation"]].iterrows():
        row[cls] = segm
    train_.append(row)
df_train_aggr = pd.DataFrame(train_)
display(df_train_aggr.head())

In [ ]:
DATASET_IMAGES = "dataset/images"
DATASET_SEGMS = "dataset/segms"

for rdir in (DATASET_IMAGES, DATASET_SEGMS):
    for sdir in ("train", "val"):
        os.makedirs(os.path.join(rdir, sdir))

In [ ]:
df_train_aggr['Case_Day'] = [f"case{r['Case']}_day{r['Day']}" for _, r in df_train_aggr.iterrows()]

CASES_DAYS = list(df_train_aggr['Case_Day'].unique())
VAL_SPLIT = 0.1
VAL_CASES_DAYS = CASES_DAYS[-int(VAL_SPLIT * len(CASES_DAYS)):]

print(f"all case-day: {len(CASES_DAYS)}")
print(f"val case-day: {len(VAL_CASES_DAYS)}")

In [ ]:
df_train_aggr.columns = ['id','class_type','segmentation','Case','Day','Slice','image','image_path','height','width','large_bowel','small_bowel','stomach','Case_Day']

In [ ]:
cat_ids = {name:id+1 for id, name in enumerate(df_train_aggr.class_type.unique())} 
cats =[{'name':name, 'id':id} for name,id in cat_ids.items()]

In [ ]:
# From https://neuniquedev.com/encode-numpy-array-using-uncompressed-rle-for-coco-dataset
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [ ]:
import shutil
import numpy as np
from PIL import Image
from joblib import Parallel, delayed
from kaggle_imsegm.mask import rle_decode

LABELS = sorted(df_train["class"].unique())

def _process_row(
    train_df, val_cases_days=VAL_CASES_DAYS, labels=LABELS,
    dir_data=DATASET_FOLDER, dir_imgs=DATASET_IMAGES, dir_segm=DATASET_SEGMS):
    val_annotations=[]
    val_images = []
    train_annotations=[]
    train_images = []       
    
    for idx, row in tqdm(train_df.iterrows()):
        
        case_day = f"case{row['Case']}_day{row['Day']}"

        
        if case_day in val_cases_days:
            sdir = 'val'

            img_dirs = os.path.dirname(row["image_path"]).split(os.path.sep)
            img_name = f"{img_dirs[-2]}_{row['image']}"
            img_path = os.path.join(dir_data, row["image_path"])
            img_path_local = os.path.join(dir_imgs, sdir, img_name)
            shutil.copy(img_path, img_path_local)
            img_size = plt.imread(img_path).shape
            
            val_images.append({'id':row.id, 'width':row.width, 'height':row.height, 'file_name':f'{img_path_local}'})
                               
            mask = np.zeros(img_size, dtype=np.uint8)
            for lb, col in enumerate(labels):
                rle = row[col]
                if not rle or not isinstance(rle, str):
                    continue
                mk = rle_decode(rle, img=mask, label=lb + 1)
                ys, xs = np.where(mk)
                x1, x2 = min(xs), max(xs)
                y1, y2 = min(ys), max(ys)
                enc =binary_mask_to_rle(mk)
                seg = {
                    'segmentation':enc, 
                    'bbox': [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
                    'area': int(np.sum(mk)),
                    'image_id':row.id, 
                    'category_id':cat_ids[row.class_type], 
                    'iscrowd':0, 
                    'id':f'{idx}_{lb}'
                }
                segm_path = os.path.join(dir_segm, sdir, img_name)
                Image.fromarray(mk).save(segm_path)
                val_annotations.append(seg)
            
        else:
            
            sdir = 'train'

            img_dirs = os.path.dirname(row["image_path"]).split(os.path.sep)
            img_name = f"{img_dirs[-2]}_{row['image']}"
            img_path = os.path.join(dir_data, row["image_path"])
            img_path_local = os.path.join(dir_imgs, sdir, img_name)
            shutil.copy(img_path, img_path_local)
            img_size = plt.imread(img_path).shape
            
            train_images.append({'id':row.id, 'width':row.width, 'height':row.height, 'file_name':f'{img_path_local}'})
                               
            mask = np.zeros(img_size, dtype=np.uint8)
            for lb, col in enumerate(labels):
                rle = row[col]
                if not rle or not isinstance(rle, str):
                    continue
                mk = rle_decode(rle, img=mask, label=lb + 1)
                ys, xs = np.where(mk)
                x1, x2 = min(xs), max(xs)
                y1, y2 = min(ys), max(ys)
                enc =binary_mask_to_rle(mk)
                seg = {
                    'segmentation':enc, 
                    'bbox': [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
                    'area': int(np.sum(mk)),
                    'image_id':row.id, 
                    'category_id':cat_ids[row.class_type], 
                    'iscrowd':0, 
                    'id':f'{idx}_{lb}'
                }
                segm_path = os.path.join(dir_segm, sdir, img_name)
                Image.fromarray(mk).save(segm_path)
                train_annotations.append(seg)            
            
    return val_annotations, train_annotations, val_images, train_images


In [ ]:
import json,itertools
val_coco, train_coco, val_imgs, train_imgs = _process_row(df_train_aggr)

In [ ]:
train_coco_json = {'categories':cats, 'images':train_imgs,'annotations':train_coco}
val_coco_json = {'categories':cats, 'images':val_imgs,'annotations':val_coco}

In [ ]:
with open('gi_tract_train_annotations_coco.json', 'w', encoding='utf-8') as f:
    json.dump(train_coco_json, f, ensure_ascii=True, indent=4)
    
with open('gi_tract_val_annotations_coco.json', 'w', encoding='utf-8') as f:
    json.dump(val_coco_json, f, ensure_ascii=True, indent=4)